In [0]:
%pip install --upgrade databricks-sdk>=0.61.0
%restart_python

In [0]:
%run ./0-Parameters

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.database import DatabaseInstance, SyncedDatabaseTable, SyncedTableSpec, NewPipelineSpec, SyncedTableSchedulingPolicy
from databricks.sdk.errors.platform import NotFound

w = WorkspaceClient()

In [0]:
try:
    instance = w.database.get_database_instance(INSTANCE_NAME)
    print(f"Database instance {INSTANCE_NAME} already exists")
except NotFound:
    instance = w.database.create_database_instance(
        DatabaseInstance(name=INSTANCE_NAME, capacity="CU_1")
    )
    print(f"Created database instance: {instance.name}")

In [0]:
sdb = SyncedDatabaseTable(
    name=SYNCED_TABLE_FULL_NAME_PG,
    database_instance_name=INSTANCE_NAME,
    logical_database_name=INSTANCE_NAME,
    spec=SyncedTableSpec(
        create_database_objects_if_missing=True,
        new_pipeline_spec=NewPipelineSpec(),
        primary_key_columns=["s", "p"],
        scheduling_policy=SyncedTableSchedulingPolicy.SNAPSHOT,
        timeseries_key="timestamp",
        source_table_full_name=TRIPLE_TABLE_FULL_NAME_UC,
    ),
)

try:
    w.database.get_synced_database_table(sdb.name)
    print(f"Synced table {sdb.name} already exists")
except NotFound:
    w.database.create_synced_database_table(sdb)
    print(f"Created synced table {sdb.name}")